<a href="https://colab.research.google.com/github/Pmilivojevic/PyTorch/blob/main/feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -U dtale
# !pip install -U six
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
import dtale
import dtale.app as dtale_app
# dtale_ap.USE_NGROK = True
dtale_app.USE_COLAB = True

pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.max_rows', None)

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/ExploratoryDataAnalysis/datasets/train.csv')
df = sns.load_dataset('planets')

In [ ]:
dtale.show(dataset)

/usr/local/lib/python3.10/dist-packages/dtale/views.py:789: FutureWarning:

['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.



https://og97w0ql39f-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/1

In [ ]:
features_nan = [
    feature for feature in dataset.columns
    if dataset[feature].isnull().sum() > 0 and
    dataset[feature].dtype == 'O'
]

for feature in features_nan:
  print(f'{feature}: {np.round(dataset[feature].isnull().mean(), 4)}')

In [ ]:
def replace_nan_vals(dataset, nan_features):
  data = dataset.copy()

  data[nan_features] = data[nan_features].fillna('Missing')

  return data

In [ ]:
dataset = replace_nan_vals(dataset, features_nan)
dataset[features_nan].isnull().sum()

In [ ]:
features_num_nan = [
    feature for feature in dataset.columns
    if dataset[feature].isnull().sum() > 0 and
    dataset[feature].dtype != 'O'
]

for feature in features_num_nan:
  print(f'{feature} {np.round(dataset[feature].isnull().mean(), 4)}: missing values')

In [ ]:
for feature in features_num_nan:
  median_val = dataset[feature].median()

  dataset[feature + '_nan'] = np.where(dataset[feature].isnull(), 1, 0)
  dataset[feature] = median_val

dataset[features_num_nan].isnull().sum()

In [ ]:
temp_features = [
    feature for feature in dataset.columns
    if ('Yr' in feature or 'Year' in feature) and
    'Sold' not in feature and 'nan' not in feature
]

for feature in temp_features:
  dataset[feature] = dataset['YrSold'] - dataset[feature]

dataset[temp_features].head()

In [ ]:
num_features=['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']

for feature in num_features:
  dataset[feature] = np.log(dataset[feature])

In [ ]:
categorical_features = [
    feature for feature in dataset.columns
    if dataset[feature].dtype == 'O'
]

In [ ]:
for feature in categorical_features:
  temp = dataset.groupby(feature)['SalePrice'].count()/len(dataset)
  temp_df = temp[temp > 0.01].index
  dataset[feature] = np.where(dataset[feature].isin(temp_df), dataset[feature], 'Rare_var')

In [ ]:
dataset[categorical_features[-1]]

In [ ]:
for feature in categorical_features:
  labels_ordered = dataset.groupby(feature)['SalePrice'].mean().sort_values().index
  labels_ordered = {k:i for i,k in enumerate(labels_ordered)}

  dataset[feature] = dataset[feature].map(labels_ordered)

In [ ]:
features_scale = [feature for feature in dataset.columns if feature not in ['Id', 'SalePrice']]
scaler = MinMaxScaler()
scaler.fit(dataset[features_scale])

In [ ]:
scaler.transform(dataset[features_scale])

In [ ]:
data = pd.concat(
    [
        dataset[['Id', 'SalePrice']].reset_index(drop=True),
        pd.DataFrame(scaler.transform(dataset[features_scale]), columns=features_scale)
    ], axis=1
)

data.head()

In [ ]:
Y_train = data[['SalePrice']]
X_train = data.drop(['Id', 'SalePrice'], axis=1)

In [ ]:
feature_select_model = SelectFromModel(Lasso(alpha=0.005, random_state=0))
feature_select_model.fit(X_train, Y_train)

In [ ]:
selected_features = X_train.columns[(feature_select_model.get_support())]

In [ ]:
X_train[selected_features].head()

,MSSubClass,MSZoning,Neighborhood,OverallQual,YearRemodAdd,RoofStyle,BsmtQual,BsmtExposure,HeatingQC,CentralAir,1stFlrSF,GrLivArea,BsmtFullBath,KitchenQual,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,PavedDrive,SaleCondition
0,0.235294,0.75,0.636364,0.666667,0.098361,0.0,0.75,0.25,1.00,1.0,0.356155,0.577712,0.333333,0.666667,0.000000,0.2,0.8,0.666667,0.50,1.0,0.75
1,0.000000,0.75,0.500000,0.555556,0.524590,0.0,0.75,1.00,1.00,1.0,0.503056,0.470245,0.000000,0.333333,0.333333,0.6,0.8,0.666667,0.50,1.0,0.75
2,0.235294,0.75,0.636364,0.666667,0.114754,0.0,0.75,0.50,1.00,1.0,0.383441,0.593095,0.333333,0.666667,0.333333,0.6,0.8,0.666667,0.50,1.0,0.75
3,0.294118,0.75,0.727273,0.666667,0.606557,0.0,0.50,0.25,0.75,1.0,0.399941,0.579157,0.333333,0.666667,0.333333,0.8,0.4,0.333333,0.75,1.0,0.00
4,0.235294,0.75,1.000000,0.777778,0.147541,0.0,0.75,0.75,1.00,1.0,0.466237,0.666523,0.333333,0.666667,0.333333,0.6,0.8,0.666667,0.75,1.0,0.75
